In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2
import motors

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
from ultralytics import YOLO

# full model
model = YOLO("yolo11l_half.engine")

# human only model
# model = YOLO('best.pt')


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [ ]:
class HistogramPersonIdentifier:
    """
    Simple HSV-histogram-based person re-identification.
    Keeps an in-memory DB of person_id -> histogram.
    """

    def __init__(self, similarity_thresh=0.6, h_bins=16, s_bins=16):
        self.person_db = {}   # person_id -> hist
        self.next_id = 0
        self.similarity_thresh = similarity_thresh
        self.h_bins = h_bins
        self.s_bins = s_bins

    def _get_histogram(self, roi_bgr):
        """Compute normalised HSV histogram for a ROI."""
        hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist(
            [hsv], [0, 1], None,
            [self.h_bins, self.s_bins],
            [0, 180, 0, 256]
        )
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist

    def _match_person(self, hist):
        """Return best-matching person ID or None."""
        best_id = None
        best_score = -1

        for pid, ref_hist in self.person_db.items():
            score = cv2.compareHist(ref_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score = score
                best_id = pid

        if best_score > self.similarity_thresh:
            return best_id
        return None

    def assign_ids(self, frame, detections):
        """
        For each person box, assign a stable ID based on histogram.
        Returns list of (x1, y1, x2, y2, person_id).
        """
        results = []

        for box in detections.boxes:
            cls = int(box.cls[0])
            if cls != 0:
                continue  # only PERSON class

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            roi = frame[y1:y2, x1:x2]

            if roi.size == 0:
                continue

            hist = self._get_histogram(roi)
            pid = self._match_person(hist)

            # New identity if no good match
            if pid is None:
                pid = self.next_id
                self.person_db[pid] = hist
                self.next_id += 1

            results.append((x1, y1, x2, y2, pid))

        return results

In [3]:
import numpy as np

identifier = HistogramPersonIdentifier(similarity_thresh=0.6)

def detect_human_from_results(results, color_frame, depth_frame, width, height):
    global identifier
    
    min_distance = float('inf')
    best_bbox = None
    best_center = None

    # for result in results:
    #     for i, bbox in enumerate(result.boxes.xyxy):
    #         cx = int((bbox[0] + bbox[2]) / 2)
    #         cy = int((bbox[1] + bbox[3]) / 2)

    #         # Depth lookup bounds check
    #         if 0 <= cx < width and 0 <= cy < height:
    #             distance = depth_frame[cy, cx]

    #             if not np.isnan(distance) and distance > 0:
    #                 if distance < min_distance:
    #                     min_distance = distance
    #                     best_bbox = bbox
    #                     best_center = [cx, cy]

    tracks = identifier.assign_ids(frame, yolo_result)

    x1, y1, x2, y2, pid = tracks[0]
    
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)

    best_bbox = [x1, x2, y1, y2]
    best_center = [cx, cy]

    return best_bbox is not None, min_distance, best_bbox, best_center

In [4]:
def pixel_to_angle(found, center, width, height, hfov, vfov):

    if not found:
        return 0.0, 0.0

    px = center[0]
    py = center[1]

    cx, cy = width / 2, height / 2

    h_angle = ((px - cx) / cx) * (hfov / 2)
    v_angle = ((py - cy) / cy) * (vfov / 2)

    return h_angle, v_angle

In [5]:
from camera import Camera 
from springfollower import SpringFollower

#create a camera object
cam = Camera()
cam.start() # start capturing the data

#create follower
follower = SpringFollower(speed=0.0, follow_distance=5.0, stopping_distance=2.0, v_max = 0.5, deadband=0.2)

[2025-11-21 10:50:36 UTC][ZED][INFO] Logging level INFO
[2025-11-21 10:50:36 UTC][ZED][INFO] Logging level INFO
[2025-11-21 10:50:36 UTC][ZED][INFO] Logging level INFO
[2025-11-21 10:50:37 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-11-21 10:50:38 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-11-21 10:50:38 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-11-21 10:50:38 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-11-21 10:50:38 UTC][ZED][INFO] [Init]  Serial Number: S/N 35129214


In [8]:
import time
import cv2

display_camera = True

run_time = range(100)
start_time = time.time()
yolo_interval = 100


latest_color_frame = None
latest_depth_frame = None

latest_results = None

# =========== Main Loop ===========
for i in run_time:
    # get current frame
    latest_color_frame, latest_depth_frame = cam.get_frame()

    # prin("
    # detect humans
    if i % yolo_interval == 0:
        print("We YOLO")
        # Confidence threshold to filter out noise.
        latest_results = model(latest_color_frame, classes=[0], verbose=False, conf=0.5)
        
    found, dist, bbox, center = detect_human_from_results(
        latest_results, latest_color_frame, latest_depth_frame, cam.width, cam.height
        )
    
    # calculate angle deviation from center
    h_angle, v_angle = pixel_to_angle(
        found, center, cam.width, cam.height, cam.hfov, cam.vfov
    )

    #print(h_angle)
    follower.update(found, h_angle, dist)

    # =========== Display images ===========
    
    if display_camera:
        # Prepare depth colormap for display
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(latest_depth_frame, alpha=0.03), 
            cv2.COLORMAP_JET)
        scale = 0.1
        resized_color = cv2.resize(latest_color_frame, None, fx=scale, fy=scale, 
                                   interpolation=cv2.INTER_AREA)
        resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                                   interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_color)
        display_depth.value = bgr8_to_jpeg(resized_depth)



end_time = time.time()
elapsed_time = end_time - start_time

print("Full model inference on ", run_time," frames completed.")
print(f"Elapsed Time: {elapsed_time} seconds")

control_frequency = len(run_time)/elapsed_time
print(f"Control frequency = ", control_frequency)

We YOLO
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HUman detected
HU

In [9]:
follower.stop()
cam.stop()